# Compute S

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import os
from glob import glob

import matplotlib.pyplot as plt

import histlib.matchup as match
import histlib.diagnosis as diag
from histlib.cstes import labels, zarr_dir, matchup_dir, var
from histlib.matchup import _data_var, _stress_var, _aviso_var

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=20, processes=20, walltime='04:00:00')
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.21:8787/status,
Dashboard: http://10.148.0.21:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.21:46305,Workers: 0
Dashboard: http://10.148.0.21:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Choose DL

In [8]:
DL =25*1e3 #meters
DT = 0.5*3600 #seconds

# Function & test

In [31]:
DL =25e3 #meters
DT = 0.5*3600 #seconds

#IF restricted computation
wd_x= ['es_cstrio_z15_alti_wd_x', 'es_cstrio_z15_drifter_wd_x']
wd_y = []
grad_x = ['alti_ggx_adt_filtered',
          'alti_ggx_adt_filtered_ocean_tide',
          'alti_ggx_adt_filtered_ocean_tide_internal_tide',
          'alti_ggx_adt_filtered_ocean_tide_internal_tide_dac',
         'aviso_alti_ggx_adt',
          'aviso_drifter_ggx_adt'
         ]
grad_y =[]
cutoff=[0, 0.1, 0.2, 0.5, 1, 1.5, 2, 2.5]

var = wd_x+grad_x+grad_y+['drifter_acc_x_0','drifter_coriolis_x_0']


def ms_dataset(dsm, l, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff) :
    dsm = match.add_except_sum(dsm, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)   
    nb = dsm.dims["obs"]
    ds = (dsm**2).mean('obs')
    ds["nb_coloc"] = nb
    ds['drifter_sat_year']=l
    ds = ds.expand_dims('drifter_sat_year')
    ds = ds.set_coords('drifter_sat_year')
    
    if "id_comb" in dsm:
        ds["id_comb"] = dsm["id_comb"]
    ## ATTRS
    for v in dsm.keys() :
        ds[v].attrs =dsm[v].attrs
        ds[v].attrs['units']=r'$m^2.s^{-4}$'
        
    return ds

def mean_dataset(dsm, l, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff) :
    dsm = match.add_except_sum(dsm, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)   
    nb = dsm.dims["obs"]
    ds = dsm.mean('obs')
    ds["nb_coloc"] = nb
    ds['drifter_sat_year']=l
    ds = ds.expand_dims('drifter_sat_year')
    ds = ds.set_coords('drifter_sat_year')
    
    if "id_comb" in dsm:
        ds["id_comb"] = dsm["id_comb"]
    ## ATTRS
    for v in dsm.keys() :
        ds[v].attrs =dsm[v].attrs
        ds[v].attrs['units']=r'$m.s^{-2}$'
        
    return ds

def corr_dataset(dsm, l, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff) :
    ds_ = match.product_dataset(dsm, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)   
    nb = ds_.dims["obs"]
    ds = ds_.mean('obs')
    ds["nb_coloc"] = nb
    ds['drifter_sat_year']=l
    ds = ds.expand_dims('drifter_sat_year')
    ds = ds.set_coords('drifter_sat_year')

    ## ATTRS
    for v in ds_.keys() :
        ds[v].attrs =ds_[v].attrs
        ds[v].attrs['units']=r'$m^2.s^{-4}$'
        
    return ds

In [9]:
l = labels[0]
dsm = xr.open_dataset(os.path.join(matchup_dir, f'matchup_{l}.zarr'))[var+['drogue_status', 'alti___distance', 'alti___time_difference']].dropna('obs').chunk({'obs':500})
# add filtered acc and coriolis
zarr_cutoff = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+".zarr")
zarr_cutoff1 = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+"_2.zarr")
if os.path.isdir(zarr_cutoff):
    dsm = xr.merge([dsm, xr.open_dataset(zarr_cutoff),xr.open_dataset(zarr_cutoff1)], join='inner').chunk({'obs':1000})
    
dsm = dsm.where(dsm.alti___distance<=DL, drop=True)
dsm = dsm.where(dsm.alti___time_difference<=DT, drop=True).drop(['alti___distance', 'alti___time_difference']).dropna('obs')
dsmd = dsm.where(dsm.drogue_status, drop=True).drop('drogue_status')
dsmnd = dsm.where(np.logical_not(dsm.drogue_status), drop=True).drop('drogue_status')
dsm = dsm.drop('drogue_status')
dsm

<xarray.Dataset>
Dimensions:                                             (obs: 4578)
Coordinates:
  * obs                                                 (obs) int64 10 ... 43032
    lat                                                 (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    lon                                                 (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    time                                                (obs) datetime64[ns] dask.array<chunksize=(143,), meta=np.ndarray>
Data variables: (12/51)
    drifter_acc_x_0                                     (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    drifter_acc_y_0                                     (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    drifter_coriolis_x_0                                (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    drifter_coriolis_y_0                                (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    e5_cstrio_z0_alti_wd_x                              (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    e5_cstrio_z0_drifter_wd_x                           (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    ...                                                  ...
    drifter_coriolis_x_1                                (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    drifter_coriolis_x_2                                (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    drifter_acc_x_15                                    (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    drifter_acc_x_25                                    (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    drifter_coriolis_x_15                               (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
    drifter_coriolis_x_25                               (obs) float64 dask.array<chunksize=(143,), meta=np.ndarray>
Attributes:
    __id:                   CMEMS-L3-SLA-Jason-3__gps
    __time_coverage_end:    2020-05-01T00:00:00
    __time_coverage_start:  2020-01-01T00:00:00

In [8]:
dsm

<xarray.Dataset>
Dimensions:                                             (obs: 4595)
Coordinates:
  * obs                                                 (obs) int64 10 ... 43032
    lat                                                 (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    lon                                                 (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    time                                                (obs) datetime64[ns] dask.array<chunksize=(142,), meta=np.ndarray>
Data variables: (12/24)
    es_cstrio_z15_alti_wd_x                             (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    es_cstrio_z15_drifter_wd_x                          (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    alti_ggx_adt_filtered                               (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    alti_ggx_adt_filtered_ocean_tide                    (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    alti_ggx_adt_filtered_ocean_tide_internal_tide      (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    alti_ggx_adt_filtered_ocean_tide_internal_tide_dac  (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    ...                                                  ...
    drifter_coriolis_x_1                                (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    drifter_coriolis_x_2                                (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    drifter_acc_x_15                                    (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    drifter_acc_x_25                                    (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    drifter_coriolis_x_15                               (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
    drifter_coriolis_x_25                               (obs) float64 dask.array<chunksize=(142,), meta=np.ndarray>
Attributes:
    __id:                   CMEMS-L3-SLA-Jason-3__gps
    __time_coverage_end:    2020-05-01T00:00:00
    __time_coverage_start:  2020-01-01T00:00:00

In [15]:
ds = ms_dataset(dsm, l)

In [16]:
ds.nb_coloc

<xarray.DataArray 'nb_coloc' (drifter_sat_year: 1)>
array([4595])
Coordinates:
  * drifter_sat_year  (drifter_sat_year) <U16 'gps_Jason-3_2020'

__________
# For all

In [ ]:
for l in labels : 
    dsm = xr.open_dataset(os.path.join(matchup_dir, f'matchup_{l}.zarr'))[var+['drogue_status', 'alti___distance', 'alti___time_difference']].dropna('obs').chunk({'obs':500})
    # add filtered acc and coriolis
    zarr_cutoff = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+"_2.zarr")
    if isdir(zarr_cutoff):
        dsm = xr.merge([dsm, xr.open_dataset(zarr_cutoff).chunk({'obs':500})], join='inner')
    dsm = dsm.where(dsm.alti___distance<=DL, drop=True)
    dsm = dsm.where(dsm.alti___time_difference<=DT, drop=True).drop(['alti___distance', 'alti___time_difference'])
    dsmd = dsm.where(dsm.drogue_status, drop=True).drop('drogue_status')
    dsmnd = dsm.where(np.logical_not(dsm.drogue_status), drop=True).drop('drogue_status')
    dsm = dsm.drop('drogue_status')
    if l==labels[0] : 
        ms_dataset(dsm, l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{int(DL//1000)}_{DT}.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
        if dsmd.dims['obs']!=0 : 
            ms_dataset(dsmd,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{int(DL//1000)}_{DT}_drogued.zarr'), encoding={'drifter_sat_year':{'dtype':'U32'}},mode='w')
        if dsmnd.dims['obs']!=0 : 
            ms_dataset(dsmnd,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{int(DL//1000)}_{DT}_undrogued.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
    else : 
        ms_dataset(dsm,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{int(DL//1000)}_{DT}.zarr'), append_dim='drifter_sat_year')
        if dsmd.dims['obs']!=0 : 
            ms_dataset(dsmd,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{int(DL//1000)}_{DT}_drogued.zarr'), append_dim='drifter_sat_year')
        if dsmnd.dims['obs']!=0 : 
            ms_dataset(dsmnd,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{int(DL//1000)}_{DT}_undrogued.zarr'),append_dim='drifter_sat_year')
    print(l)


gps_Jason-3_2020
argos_Jason-3_2020
gps_SARAL_2020
argos_SARAL_2020
gps_Cryosat-2_2020
argos_Cryosat-2_2020


________
# Run gps_sentinel_2019

In [5]:
l='gps_Sentinel-3_A_2019'

DL =25e3 #meters
DT = 0.5*3600 #seconds

#IF restricted computation
wd_x= ['es_cstrio_z15_alti_wd_x', 'es_cstrio_z15_drifter_wd_x']
wd_y = []
grad_x = ['alti_ggx_adt_filtered',
          'alti_ggx_adt_filtered_ocean_tide',
          'alti_ggx_adt_filtered_ocean_tide_internal_tide',
          'alti_ggx_adt_filtered_ocean_tide_internal_tide_dac',
         'aviso_alti_ggx_adt',
          'aviso_drifter_ggx_adt'
         ]
grad_y =[]
cutoff=[0, 0.1, 0.2, 0.5, 1, 1.5, 2, 2.5]

var = wd_x+grad_x+grad_y+['drifter_acc_x_0','drifter_coriolis_x_0']


def ms_dataset(dsm, l, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff) :
    dsm = match.add_except_sum(dsm, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)   
    nb = dsm.dims["obs"]
    ds = (dsm**2).mean('obs')
    ds["nb_coloc"] = nb
    ds['drifter_sat_year']=l
    ds = ds.expand_dims('drifter_sat_year')
    ds = ds.set_coords('drifter_sat_year')
    
    if "id_comb" in dsm:
        ds["id_comb"] = dsm["id_comb"]
    ## ATTRS
    for v in dsm.keys() :
        ds[v].attrs =dsm[v].attrs
        ds[v].attrs['units']=r'$m^2.s^{-4}$'
        
    return ds

def mean_dataset(dsm, l, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff) :
    dsm = match.add_except_sum(dsm, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)   
    nb = dsm.dims["obs"]
    ds = dsm.mean('obs')
    ds["nb_coloc"] = nb
    ds['drifter_sat_year']=l
    ds = ds.expand_dims('drifter_sat_year')
    ds = ds.set_coords('drifter_sat_year')
    
    if "id_comb" in dsm:
        ds["id_comb"] = dsm["id_comb"]
    ## ATTRS
    for v in dsm.keys() :
        ds[v].attrs =dsm[v].attrs
        ds[v].attrs['units']=r'$m.s^{-2}$'
        
    return ds

def run_ms_global(l):

    dsm = xr.open_dataset(os.path.join(matchup_dir, f'matchup_{l}.zarr'))[var+['drogue_status', 'alti___distance', 'alti___time_difference']].dropna('obs').chunk({'obs':500})
    # add filtered acc and coriolis
    zarr_cutoff = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+".zarr")
    zarr_cutoff1 = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+"_2.zarr")
    if os.path.isdir(zarr_cutoff):
        dsm = xr.merge([dsm, xr.open_dataset(zarr_cutoff),xr.open_dataset(zarr_cutoff1)], join='inner').chunk({'obs':1000})
        
    dsm = dsm.where(dsm.alti___distance<=DL, drop=True)
    dsm = dsm.where(dsm.alti___time_difference<=DT, drop=True).drop(['alti___distance', 'alti___time_difference']).dropna('obs')
    dsmd = dsm.where(dsm.drogue_status, drop=True).drop('drogue_status')
    dsmnd = dsm.where(np.logical_not(dsm.drogue_status), drop=True).drop('drogue_status')
    dsm = dsm.drop('drogue_status')
    return dsm, dsmd, dsmnd

dsm, dsmd, dsmnd = run_ms_global(l)

In [6]:
ms_dataset(dsm, l).to_zarr(os.path.join(zarr_dir+'_ok','global',f'ms_{int(DL//1000)}_{DT}_{l}.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
mean_dataset(dsm, l).to_zarr(os.path.join(zarr_dir+'_ok','global',f'mean_{int(DL//1000)}_{DT}_{l}.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')

In [8]:
ms_dataset(dsmd, l).to_zarr(os.path.join(zarr_dir+'_ok','global',f'ms_{int(DL//1000)}_{DT}_{l}_drogued.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
mean_dataset(dsmd, l).to_zarr(os.path.join(zarr_dir+'_ok','global',f'mean_{int(DL//1000)}_{DT}_{l}_drogued.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')

In [9]:
ms_dataset(dsmnd, l).to_zarr(os.path.join(zarr_dir+'_ok','global',f'ms_{int(DL//1000)}_{DT}_{l}_undrogued.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
mean_dataset(dsmnd, l).to_zarr(os.path.join(zarr_dir+'_ok','global',f'mean_{int(DL//1000)}_{DT}_{l}_undrogued.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')

________
# Open test 

In [33]:
zarr

'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/ms_25_1800.0.zarr'

In [20]:
DL =25*1e3 #meters
DT = int(1800) #seconds
zarr = zarr_dir+'_ok/'+'mean_25_1800.0_2.zarr'
ds = xr.open_dataset(zarr)
ds.to_zarr(zarr_dir+'_ok/'+'mean_25_1800.0.zarr', append_dim='drifter_sat_year')

In [25]:
zarr = zarr_dir+'_ok/'+'ms_25_1800.0.zarr'
ds0 = xr.open_dataset(zarr)
ds0_ = ds0.isel(drifter_sat_year=-1).compute()
ds0_

<xarray.Dataset>
Dimensions:                                                                                   (
                                                                                               id_comb: 60)
Coordinates:
    drifter_sat_year                                                                          <U32 ...
  * id_comb                                                                                   (id_comb) <U75 ...
Data variables: (12/331)
    alti_ggx_adt_filtered                                                                     float64 ...
    alti_ggx_adt_filtered_ocean_tide                                                          float64 ...
    alti_ggx_adt_filtered_ocean_tide_internal_tide                                            float64 ...
    alti_ggx_adt_filtered_ocean_tide_internal_tide_dac                                        float64 ...
    aviso_alti_ggx_adt                                                                        float64 ...
    aviso_alti_ggy_adt                                                                        float64 ...
    ...                                                                                        ...
    sum_co__2__adt_filtered_ocean_tide__es_cstrio_z15__alti_x                                 float64 ...
    sum_co__2__adt_filtered_ocean_tide__es_cstrio_z15__drifter_x                              float64 ...
    sum_co__2__adt_filtered_ocean_tide_internal_tide__es_cstrio_z15__alti_x                   float64 ...
    sum_co__2__adt_filtered_ocean_tide_internal_tide__es_cstrio_z15__drifter_x                float64 ...
    sum_co__2__adt_filtered_ocean_tide_internal_tide_dac__es_cstrio_z15__alti_x               float64 ...
    sum_co__2__adt_filtered_ocean_tide_internal_tide_dac__es_cstrio_z15__drifter_x            float64 ...

In [8]:
zarr = zarr_dir+'_ok/'+'mean_25_1800.0_drogued.zarr'
dsm = xr.open_dataset(zarr)
dsm

<xarray.Dataset>
Dimensions:                                                                                   (
                                                                                               drifter_sat_year: 38,
                                                                                               id_comb: 60)
Coordinates:
  * drifter_sat_year                                                                          (drifter_sat_year) <U32 ...
  * id_comb                                                                                   (id_comb) <U75 ...
Data variables: (12/331)
    alti_ggx_adt_filtered                                                                     (drifter_sat_year) float64 ...
    alti_ggx_adt_filtered_ocean_tide                                                          (drifter_sat_year) float64 ...
    alti_ggx_adt_filtered_ocean_tide_internal_tide                                            (drifter_sat_year) float64 ...
    alti_ggx_adt_filtered_ocean_tide_internal_tide_dac                                        (drifter_sat_year) float64 ...
    aviso_alti_ggx_adt                                                                        (drifter_sat_year) float64 ...
    aviso_alti_ggy_adt                                                                        (drifter_sat_year) float64 ...
    ...                                                                                        ...
    sum_co__2__adt_filtered_ocean_tide__es_cstrio_z15__alti_x                                 (drifter_sat_year) float64 ...
    sum_co__2__adt_filtered_ocean_tide__es_cstrio_z15__drifter_x                              (drifter_sat_year) float64 ...
    sum_co__2__adt_filtered_ocean_tide_internal_tide__es_cstrio_z15__alti_x                   (drifter_sat_year) float64 ...
    sum_co__2__adt_filtered_ocean_tide_internal_tide__es_cstrio_z15__drifter_x                (drifter_sat_year) float64 ...
    sum_co__2__adt_filtered_ocean_tide_internal_tide_dac__es_cstrio_z15__alti_x               (drifter_sat_year) float64 ...
    sum_co__2__adt_filtered_ocean_tide_internal_tide_dac__es_cstrio_z15__drifter_x            (drifter_sat_year) float64 ...

In [9]:
dsm.alti_ggx_adt_filtered

<xarray.DataArray 'alti_ggx_adt_filtered' (drifter_sat_year: 38)>
array([-1.118149e-06, -3.732157e-07,  1.454797e-06, -7.183918e-07,
       -7.934131e-07,  7.244419e-07,  9.487175e-06, -3.377631e-08,
       -4.546172e-06,  3.511395e-07,  7.713215e-06, -6.550257e-08,
        4.895352e-07, -2.199483e-08,  5.138503e-06,  4.572777e-07,
       -1.033852e-06, -1.245979e-07, -3.444865e-08,  2.919565e-07,
        1.283529e-06, -1.952023e-07,  3.933753e-07, -2.825214e-07,
        1.467614e-06,  2.399777e-07, -6.561103e-08,  1.430484e-07,
       -1.063440e-06, -7.724592e-08,  1.202505e-06,  3.515327e-07,
        1.385043e-06, -5.191834e-07, -2.824836e-07,  1.203561e-06,
        8.541847e-07,  1.032421e-06])
Coordinates:
  * drifter_sat_year  (drifter_sat_year) <U32 'gps_Jason-3_2020' ... 'gps_Jas...
Attributes:
    comment:    The sea level anomaly is the sea surface height above mean se...
    long_name:  $g\partial_x$adt_filtered
    units:      $m.s^{-2}$

In [ ]:
cluster.close()